<a href="https://colab.research.google.com/github/divyansh1195/Deep-Learning/blob/master/Keras%20Tuner/Fashion_Mnist_Keras_Tuner_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 1.7MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=966f10f85ae05d706cd09f30daa6ba5405fd87af3f61f418c574127f9751dde1
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=0dae2fb4dc84dbadfcdc065eede44dc0c0f0c478def47d3a2f98e4befcd79974
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


# Importing the Libraries and the dataset

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)


# import the libraries
from tensorflow import keras
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, array_to_img, img_to_array
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [3]:
fashion_mnist=keras.datasets.fashion_mnist#load fashion_mnist dataset from keras

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data() # training and test dataset images

In [5]:
# since, the fashion_mnist dataset is a black and white dataset, we need to scale down the images' pixel between 0 and 1.
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape#first image

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  #hp is hyperparameters
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  #softmax activation function of a neural network to normalize the output of a network to a probability distribution over predicted output classes
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [9]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [10]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [11]:
#searcg for best parameter from build model
#initially, we will test run it only for 2 epochs with 5 trials
tuner_search.search(train_images,train_labels,epochs=2,validation_split=0.1)

Epoch 1/2
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3928 - accuracy: 0.8577 - val_loss: 0.2886 - val_accuracy: 0.8942
Epoch 2/2
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2569 - accuracy: 0.9053 - val_loss: 0.2636 - val_accuracy: 0.9047


Epoch 1/2
1688/1688 [==============================] - 8s 5ms/step - loss: 0.4111 - accuracy: 0.8538 - val_loss: 0.2987 - val_accuracy: 0.8877
Epoch 2/2
1688/1688 [==============================] - 8s 5ms/step - loss: 0.2693 - accuracy: 0.9009 - val_loss: 0.2773 - val_accuracy: 0.8985


Epoch 1/2
1688/1688 [==============================] - 11s 6ms/step - loss: 0.5251 - accuracy: 0.8228 - val_loss: 0.3777 - val_accuracy: 0.8600
Epoch 2/2
1688/1688 [==============================] - 11s 6ms/step - loss: 0.3724 - accuracy: 0.8614 - val_loss: 0.3438 - val_accuracy: 0.8737


Epoch 1/2
1688/1688 [==============================] - 9s 6ms/step - loss: 2.3111 - accuracy: 0.0979 - val_loss: 2.3053 - val_accuracy: 0.1008
Epoch 2/2
1688/1688 [==============================] - 9s 5ms/step - loss: 2.3040 - accuracy: 0.0991 - val_loss: 2.3034 - val_accuracy: 0.0985


Epoch 1/2
1688/1688 [==============================] - 10s 6ms/step - loss: 0.4951 - accuracy: 0.8281 - val_loss: 0.3885 - val_accuracy: 0.8577
Epoch 2/2
1688/1688 [==============================] - 10s 6ms/step - loss: 0.3584 - accuracy: 0.8698 - val_loss: 0.3838 - val_accuracy: 0.8630


INFO:tensorflow:Oracle triggered exit


In [12]:
model=tuner_search.get_best_models(num_models=1)[0]#best models

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 80)                1858640   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                810       
Total params: 1,888,810
Trainable params: 1,888,810
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=2)

Epoch 3/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2000 - accuracy: 0.9259 - val_loss: 0.2632 - val_accuracy: 0.9060
Epoch 4/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1574 - accuracy: 0.9402 - val_loss: 0.2657 - val_accuracy: 0.9082
Epoch 5/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.1169 - accuracy: 0.9567 - val_loss: 0.2849 - val_accuracy: 0.9143
Epoch 6/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0885 - accuracy: 0.9669 - val_loss: 0.3084 - val_accuracy: 0.9132
Epoch 7/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0683 - accuracy: 0.9748 - val_loss: 0.3536 - val_accuracy: 0.9078
Epoch 8/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0514 - accuracy: 0.9810 - val_loss: 0.3739 - val_accuracy: 0.9152
Epoch 9/10
1688/1688 [==============================] - 9s 5ms/step - loss: 0.0403 - accuracy: 0.9855 - val_loss: 0.4679 - val_accuracy: